In [34]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
)
import time 
import numpy as np


In [35]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

device = "cuda" if torch.cuda.is_available() else "cpu"

if device == "cpu":
    print("해당 작업이 CPU에서 진행됩니다.....🙊")

In [36]:
def get_gpu_memory_usage():
    # MB 단위로 출력한다.
    return torch.cuda.memory_allocated() / 1024 ** 2 

In [37]:
def meauser_inference_speed(model, tokenizer, text, num_runs=100):
    model.eval()
    times = []

    # 워밍업을 위한 단계 ( 캐쉬, 로드 등 다양한 변수를 제외하고 측정하기 위함 )
    for _ in range(num_runs):
        # return_tensors="pt"는 PyTorch를 위한 데이터를 반환해준다.
        inputs = tokenizer(text, return_tensors="pt").to(device)
        _  = model(**inputs)

    for _ in range(num_runs):
        inputs = tokenizer(text, return_tensors="pt").to(device)
        # time.perf_counter()가 성능 검증할떄 time.time()보다 세밀하다.
        start_time = time.perf_counter()

        with torch.no_grad():
            _ = model(**inputs)
        end_time = time.perf_counter()
        times.append(end_time - start_time)

    avg_time = np.mean(times) * 1000
    return avg_time
    

In [38]:
print("------🦇FP32 모델 로드 및 평가 진행🦇-----")

tokenizer = AutoTokenizer.from_pretrained(model_name)
fp32_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    device_map="auto"
)

fp32_memory = get_gpu_memory_usage()

sample_text = "This is a fantastic moive, I really enjoyed it!!"
fp32_latency = meauser_inference_speed(fp32_model, tokenizer, sample_text)

print(f" 🐗 FP32 모델 GPU 메모리 사용량: {fp32_memory} MB")
print(f" 🐗 FP32 모델 평균 추론 시간 : {fp32_latency}(second)")

------🦇FP32 모델 로드 및 평가 진행🦇-----


 🐗 FP32 모델 GPU 메모리 사용량: 610.49658203125 MB
 🐗 FP32 모델 평균 추론 시간 : 5.0102535472251475(second)


In [39]:
# 정확한 측정을 위해서 fp32 모델을 제거 합니다.
del fp32_model
torch.cuda.empty_cache()

In [40]:
%pip install -U bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [41]:
from transformers import BitsAndBytesConfig

print("----🦇INT8 모델 로드 및 평가🦇-----")

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4", # Normal Float4
    bnb_4bit_compute_dtype=torch.bfloat16 # 4비트로 저장하지만 계산시 16비트로 변환
)

int8_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)

int8_memory = get_gpu_memory_usage()
int8_latency = meauser_inference_speed(int8_model, tokenizer, sample_text)

print(f" 🐗 INT8 모델 GPU 메모리 사용량: {int8_memory} MB")
print(f" 🐗 INT8 모델 평균 추론 시간 : {int8_latency}(second)")

----🦇INT8 모델 로드 및 평가🦇-----


 🐗 INT8 모델 GPU 메모리 사용량: 375.6787109375 MB
 🐗 INT8 모델 평균 추론 시간 : 12.692619708832353(second)


In [43]:
print(f"=====🦟최종 정리🦟=====")

print(f"모델 유형 | GPU 메모리 사용량 | 평균 추론 시간 ")
print(f"FP32 | {fp32_memory} | {fp32_latency}")
print(f"INT8 | {int8_memory} (대략 {fp32_memory // int8_memory}% 절약) | {int8_latency}")

=====🦟최종 정리🦟=====
모델 유형 | GPU 메모리 사용량 | 평균 추론 시간 
FP32 | 610.49658203125 | 5.0102535472251475
INT8 | 375.6787109375 (대략 1.0% 절약) | 12.692619708832353


### 결론
Quantization을 사용하면 확실히 GPU RAM을 줄입니다. 하지만 추론 속도의 경우 양자화 및 역양자화 과정 등 오버헤드가 발생하며 오히려 연산 시간이 더 오래걸리는 것을 확인할 수 있었습니다.